## This technical report is focusing on feature extraction:
- sentiment (done)
- on the todo list: regessive imagery dictionary scores (used by 750words.com), us vs them, time orientation, certainty (pattern library), concepts (pattern library).

In [1]:
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
import json
import pandas as pd
import numpy as np
import json
import re
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline

In [74]:
private_df = pd.read_csv("/Users/KerryChowChow/DSI-SF-3-kelly/CAPSTONE/csv/fb_private.csv")

In [58]:
private_df[private_df.length > 10000]
#whoa, so the longest string was 41885 words

,Unnamed: 0,old_thread,user,message,year,month,hour,length,type,new_thread,talking_to
84817,84817,2391,Person_1288,"From July 31st to August 2nd, the Google HQ in...",2015,8,9,10419,private,1227.0,Person_1288
100485,100485,2791,Main_User,Repertoire as menu \nThe language of physical ...,2016,1,8,11553,private,1529.0,Person_1604
138803,138803,2885,Person_506,/*\n YOU $Z$$$ZZ$$$$$77$7777777777777777777...,2016,4,15,11455,private,NaN,NaN
356942,356942,6955,Main_User,7/24/10\n\nOrtho-Core multivitamin- Vitamins a...,2013,4,23,41885,private,NaN,NaN
357053,357053,6955,Person_1883,Zhenya Pavlov\nhey can you hit me back with ev...,2013,3,18,11718,private,NaN,NaN


In [71]:
test_str2 = private_df.iloc[84817,:].message

In [72]:
test_str2 = test_str2.decode('utf-8')


## Create sentiment tags for each sentence
### textblob has 2 types of analyzer. Naivebayes was trained on movie reviews and twitter.
#### NaiveBays was really slow and Pattern library was a lot faster. It's unclear what model Pattern is using though and couldn't find it on their website

The polarity score is a float within the range [-1.0, 1.0]. The subjectivity is a float within the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective.

In [68]:
blob = TextBlob(test_str2)
blob.sentiment
#trying on one sentence and it seems like it's taking a long time. Takes at least 10 seconds.
# like the most common NLP problem, the problem for a bag of words sentiment analysis
# is that it doesn't capture the meaning of a whole sentence

Sentiment(polarity=0.2011938208958077, subjectivity=0.48187181655724715)

After testing over 10 different strings and string length from 3 to 1000. I am subjectively and possibly biased to say that I am positive about the Pattern library sentiment analysis.

I think, to get more granular sentiment analysis, it's better to break down those long message into sentences to get a mix of differetn sentiments

## Applying TextBlob sentiment to the dataset

In [81]:
#decode the unicode
message = private_df.message.apply(lambda x: x.decode('utf-8'))

In [83]:
sentiment_m = message.map(lambda x: TextBlob(x).sentiment)

In [89]:
sentiment_df = pd.DataFrame(sentiment_m.tolist())

In [95]:
whole_senti_df = pd.merge(private_df,sentiment_df, how = 'left', left_index= True, right_index= True)

In [115]:
whole_senti_df[(whole_senti_df.subjectivity == 0.0 )& (whole_senti_df.polarity > 0.4)]

,Unnamed: 0,thread#,user,message,Year,Month,Hour,Length,convo_style,polarity,subjectivity
245751,452432,4993,Main_User,Success spirals! Cascading syncope!,2014,2,15,35,private,0.46875,0.0


In [116]:
whole_senti_df.to_csv("fb_senti.csv", encoding='utf-8')

## Looking at how accurate the polarity is
examples when it's not very accurate:
- "Lets touch base. Have a million ideas to share!"
- "Damn. This is insane!"
- "you can force people to create an evil ai"
- "Worst case I just bring your food up to you."
- "Hey! Let's touch base for tonight! "
- "yeah this Sunday is the rub. That is awful soon!"

examples of decent prediction:
- "yeah I think crime is terrible in SF"
- "U r insulting me when u r sleeping around with..."
- "U know .. I know myself well.. I can be v evil"
- "Yes. The worst thing you can do, is nothing."
- "i dislike the boring repeating job"
- "Make his life miserable."